In [57]:
using Pkg
Pkg.activate("/home/fedflorio/master_thesis")

  Activating project at `~/master_thesis`


In [58]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, Printf, IndexedGraphs, Statistics, Random
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using JLD2
using TensorTrains: summary_compact
using SparseArrays;

In [59]:
T = 20
N = 30
seed = 2

c = 2
gg = erdos_renyi(N, c/N; seed)
g = IndexedGraph(gg)

λ_unif = 0.5
ρ_unif = 0.2
λ = zeros(N,N)
for i in CartesianIndices(λ)
    if !iszero(g.A[i])
        λ[i] = rand()
    end
end
λ = sparse(λ)
ρ = rand(N)
γ = 0.5

# T = 3
# N = 2
# seed = 3

# A = [0 1; 1 0]
# g = IndexedGraph(A)

# λ_unif = 0.5
# ρ_unif = 0.6
# λ = sparse(λ_unif .* A)
# # λ = sparse([0 λ_unif; 0 0])
# ρ = fill(ρ_unif, N)
# γ = 0.5

sis = SIS_heterogeneous(λ, ρ, T; γ);
bp = mpbp(sis);

In [60]:
g.A

30×30 SparseMatrixCSC{Int64, Int64} with 52 stored entries:
⎡⠀⠀⠀⠀⠈⠀⠈⠀⠈⠀⠈⠀⠠⠀⠀⎤
⎢⠀⠀⢀⠐⠐⠀⢀⠬⠀⠀⢀⠀⠠⠀⠀⎥
⎢⠂⠀⠐⠀⠠⠂⠰⠀⠈⠀⠁⡀⠀⠀⠀⎥
⎢⠂⠀⡀⡔⠐⠂⠀⠀⠀⠄⠁⠀⠀⢐⠀⎥
⎢⠂⠀⠀⠀⠂⠀⠀⠄⠀⠀⠀⠈⠀⠀⠀⎥
⎢⠂⠀⠀⠐⠁⠠⠁⠀⡀⠀⠀⠀⠀⠄⡀⎥
⎢⠀⠂⠀⠂⠀⠀⢀⢀⠀⠀⠀⠄⠀⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⎦

In [61]:
obs_times = collect(0:T)
nobs = floor(Int, N * length(obs_times) * 1.0)
obs_fraction = nobs / N
rng = MersenneTwister(seed)
X, observed = draw_node_observations!(bp, nobs, times = obs_times .+ 1, softinf=Inf; rng);

In [62]:
X

30×21 Matrix{Int64}:
 1  2  2  1  2  1  2  1  2  1  1  2  2  2  2  2  2  2  2  2  1
 2  2  2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  2  1  2  1  2  1  2  1  2  2  2  1  1  1  2  2  2  1  2  2
 2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  1  1  1
 2  1  1  1  2  1  2  1  1  1  2  1  1  2  2  1  2  1  2  2  1
 2  1  2  1  1  2  2  2  2  1  2  1  2  1  2  1  2  2  2  1  2
 2  1  2  1  2  2  1  2  1  1  1  2  1  1  2  2  2  2  2  2  1
 1  2  2  2  2  2  2  2  1  2  2  1  2  2  2  2  2  2  2  2  2
 2  1  2  1  1  1  2  1  1  1  1  2  1  1  2  1  2  1  2  1  2
 2  1  2  1  2  2  2  2  2  1  2  2  1  2  2  1  2  1  2  1  2
 1  1  1  1  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2  2
 2  2  2  2  2  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1
 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  2
 ⋮              ⋮              ⋮              ⋮              ⋮
 2  1  2  1  1  1  2  1  1  1  1  1  1  2  1  1  1  1  1  2  2
 1  1  1  1  1  1  1  1  1  1  1  

In [63]:
reset_messages!(bp)
reset_beliefs!(bp)
svd_trunc = TruncBond(5)

iters, = iterate!(bp, maxiter=5; svd_trunc, tol=1e-12);

Running MPBP: iter 2    Time: 0:00:00
  Δ:                4.440892098500626e-16
4m  SVD Matrix size:  5

In [64]:
obs_node = 1
for t in eachindex(bp.w[obs_node])
    bp.w[obs_node][t].ρ = ρ[obs_node]
end

der_mpbp = der_ρ(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

5.512529166638498

In [65]:
ϵ = 1e-8

for i in vertices(g)
    for t in eachindex(bp.w[i])
        bp.w[i][t].ρ = ρ[obs_node]
    end
end

logzᵢ0 = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

for t in eachindex(bp.w[obs_node])
    bp.w[obs_node][t].ρ += ϵ
end

logzᵢϵ = onebpiter!(bp, obs_node, eltype(bp.w[obs_node]); svd_trunc)

der_num = logzᵢϵ/ϵ - logzᵢ0/ϵ 

5.5125274658203125